## Create Agents to Research and Write an Article

Install the below libraries with their associated version:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# warnings
import warnings
warnings.filterwarnings('ignore')

Let's import Crewai libraries with its three main components

In [ ]:
# import Crewai libraries
from crewai import Agent, Task, Crew, Process, LLM
from dotenv import load_dotenv, find_dotenv
from langchain_groq import ChatGroq
from IPython.display import Markdown

load_dotenv(override=True)

True

In [41]:
model= "moonshotai/kimi-k2-instruct"
llm = LLM(model = model)

Setup the package/module scripts to be importable.

In [13]:
import os, sys

# parent directory path of the package/module "scripts" 
scripts_dir = os.path.abspath("../scripts")
# check and append "scripts_dir" in sys.path if it ain't.
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir) 

import utils

Let's set our LLM

In [14]:
# openai_api_key = utils.get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

#### Creating Agents
* Let's define our Agent and provide a `role`, `goal`, and `backstory`
* It has been seen that LLMs perform better when they are role playing.

We will create three agents namely: 
* Planner
* Writer
* Editor

##### Agent: Planner

In [29]:
planner = Agent(
    role= "Content Planner",
    goal= "Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article"
              "about the topic {topic}."
              "You collect information that helps the" 
              "audiance learn something"
              "and make informed decisions"
              "Your work is the basis for the Content Writer"
              "to write an article on this topic.",
    allow_delegation=False,
    verbose= True              
)

In [40]:
planner

Agent(role=Content Planner, goal=Plan engaging and factually accurate content on {topic}, backstory=You're working on planning a blog articleabout the topic {topic}.You collect information that helps theaudiance learn somethingand make informed decisionsYour work is the basis for the Content Writerto write an article on this topic.)

##### Agent: Writer

In [44]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [45]:
writer

Agent(role=Content Writer, goal=Write insightful and factually accurate opinion piece about the topic: {topic}, backstory=You're working on a writing a new opinion piece about the topic: {topic}. You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. You follow the main objectives and direction of the outline, as provide by the Content Planner. You also provide objective and impartial insights and back them up with information provide by the Content Planner. You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.)

##### Agent: Editor

In [46]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

#### Creating Tasks
* Let's define our Tasks and provide them a `description`, `expected_output`, and `agent`

##### Task: Plan

In [47]:
plan = Task(
    description= (
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points,"
            "and a call to action. \n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with "
                    "with an outline, audience analysis "
                    "SEO keywords, and resources",
    agent= planner,
)

In [48]:
plan

Task(description=1. Prioritize the latest trends, key players, and noteworthy news on {topic}.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points,and a call to action. 
4. Include SEO keywords and relevant data or sources., expected_output=A comprehensive content plan document with with an outline, audience analysis SEO keywords, and resources)

##### Task: write

In [49]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [50]:
write

Task(description=1. Use the content plan to craft a compelling blog post on {topic}.
2. Incorporate SEO keywords naturally.
3. Sections/Subtitles are properly named in an engaging manner.
4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.
5. Proofread for grammatical errors and alignment with the brand's voice.
, expected_output=A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.)

##### Task: edit

In [51]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [52]:
editor

Agent(role=Editor, goal=Edit a given blog post to align with the writing style of the organization. , backstory=You are an editor who receives a blog post from the Content Writer. Your goal is to review the blog post to ensure that it follows journalistic best practices,provides balanced viewpoints when providing opinions or assertions, and also avoids major controversial topics or opinions when possible.)

#### Creating the Crew
- Let's create our crew of agents
* Let's pass the tasks to be performed by those agents
- `verbose=2` allows you to see all the logs of the execution


In [57]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    llm = llm,
    # verbose=2
)

In [58]:
crew

Crew(id=ab734d9e-a269-44d5-aaff-623c35fb3415, process=sequential, number_of_agents=3, number_of_tasks=3)

#### Running the Crew
*Note*: LLMs can provide different outputs for the same input. We call fuzzy output.

In [19]:
result = crew.kickoff(inputs={"topic": "MQL5 for Expert Advisor"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on MQL5 for Expert Advisor.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points,and a call to action. 
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: MQL5 for Expert Advisor

Introduction:
- Brief overview of MQL5 and its importance in developing Expert Advisors
- Highlight the latest trends and advancements in the MQL5 platform

Key Points:
1. Latest Trends in MQL5:
- Integration of machine learning and AI in Expert Advisor development
- Increased focus on algorithmic trading strategies
- Enhanced user interface and usability features in MQL5

2. Key Players in MQL5:
- MetaQuotes Software Corp. as the developer of MQL5 platform
- Leading Expe


Display the results of your execution as markdown in the notebook.

In [20]:
from IPython.display import Markdown

Markdown(result)

# MQL5 for Expert Advisor: Revolutionizing Algorithmic Trading

## Introduction
In the realm of algorithmic trading, MQL5 serves as a cornerstone in empowering traders to create sophisticated Expert Advisors. Developed by MetaQuotes Software Corp., MQL5 is a robust programming language that enables traders to automate their trading strategies with precision. The platform has played a pivotal role in reshaping the landscape of algorithmic trading, offering advanced features and functionalities. This blog post aims to explore the latest trends and advancements in MQL5, emphasizing its significance in the development of Expert Advisors.

## Latest Trends in MQL5
A notable trend in MQL5 is the integration of machine learning and artificial intelligence in Expert Advisor development. Traders now have the opportunity to harness cutting-edge technologies to build intelligent trading systems that can adapt dynamically to market conditions. Furthermore, there is a growing emphasis on algorithmic trading strategies, leading traders to explore innovative approaches to enhance their trading performance. MQL5 has also improved its user interface and usability features, catering to traders of all levels.

## Key Players in MQL5
MetaQuotes Software Corp. stands out as the primary developer of the MQL5 platform, consistently innovating to meet the changing needs of traders. Prominent Expert Advisor developers and traders actively utilize MQL5 for trading automation, pushing the boundaries of what can be achieved in algorithmic trading. Online communities and forums dedicated to sharing MQL5 strategies and tips serve as valuable resources for traders looking to expand their expertise in the field.

## Noteworthy News on MQL5
Recent updates and releases in the MQL5 platform have introduced exciting new features and functionalities, further enhancing its capabilities. Success stories of traders leveraging MQL5 for Expert Advisor development demonstrate the platform's effectiveness in optimizing trading performance. Regulatory changes affecting the use of MQL5 in trading environments underscore the importance of staying informed and compliant in the dynamic landscape of algorithmic trading.

## Conclusion
In conclusion, MQL5 continues to drive innovation in algorithmic trading, equipping traders with advanced tools to automate their trading strategies efficiently. By keeping up with the latest trends, key players, and significant developments in the MQL5 industry, traders can unlock new opportunities and achieve greater success in their trading pursuits. Whether you are a seasoned trader seeking to automate your strategies or an enthusiast of algorithmic trading looking for advanced tools, MQL5 offers a solid platform for growth and innovation in the realm of trading automation.

*Join the MQL5 community today and discover the advantages of utilizing MQL5 for Expert Advisor development. Engage in online forums and resources to enhance your algorithmic trading skills and stay ahead of the curve in the ever-evolving world of trading automation.*

### Another Agentic AI for research

In [82]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool 
import os

# 1. setup an LLM
# llm = LLM(model="groq/moonshotai/kimi-k2-instruct", temperature=0) 
llm = LLM(model="groq/meta-llama/llama-4-maverick-17b-128e-instruct")

# 2. Agent definition
researcher_agent= Agent(
    role = "{topic} Senior Researcher", 
    goal = "Uncover groundbreaking technologies in {topic} for years 2024 and 2025",
    backstory= "Driven by curiosity, you explore and share the latest innovations and the state-of-the-art of {topic}",
    # tools = [SerperDevTool()], 
    llm = llm,
    allow_delegation=False,
    verbose= True,
)

# 3. Task definition

researcher_task = Task(
    description= "Define and explain clearly what is {topic} in formal and easy way to understand."
    "Identify the next big trend in {topic} technologies with pros and cons.",
    expected_output="A report on emerging {topic} technologies, a well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs",
    agent= researcher_agent,
)

# 4. Crew defintion or kickoff

def main():
    crew = Crew(
    agents = [researcher_agent],
    tasks = [researcher_task],
    process = Process.sequential,
    verbose = True,
    memory = True,
    )
    
    result = crew.kickoff(inputs={"topic": "AI agents"})
    # Markdown(result.raw)
    display(Markdown(result.raw))

if __name__ == "__main__":
    main()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c1e2ac8a-3131-4788-8bd6-39f6cbbb5a99                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-07-28 20:44:31,888 - 131823051266944 - rag_storage.py-rag_storage:132 - ERROR: Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-28 20:44:35,677 - 131823051266944 - rag_storage.py-rag_storage:132 - ERROR: Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭────────────────────────────────────────────── 🧠 Retrieved Memory ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Historical Data:                                                                                               │
│  - Provide more specific examples of AI agents in different industries.                                         │
│  - Include more technical details on the implementation of Multi-Agent Systems.                                 │
│  - Discuss potential future applications of advanced AI agents beyond personal assistance.                      │
│  - Discuss potential future applications of AI agents beyond current trends.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────── Retrieval Time: 9477.23ms ───────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI agents Senior Researcher                                                                             │
│                                                                                                                 │
│  Task: Define and explain clearly what is AI agents in formal and easy way to understand.Identify the next big  │
│  trend in AI agents technologies with pros and cons.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: AI agents Senior Researcher                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Thought                                                                                                     │
│  To address the task, I will first define and explain AI agents in a formal and easy-to-understand manner.      │
│  Then, I will identify the next big trend in AI agent technologies, discussing their pros and cons. I will      │
│  provide a detailed report in markdown format, including examples of AI agents across various industries,       │
│  technical details on Multi-Agent Systems, and potential future applications beyond current trends.             │
│                                                                                                                 │
│  ## Final Answer                                                                                                │
│  # AI Agents: Definition and Explanation                                                                        │
│  AI agents are software programs designed to perform specific tasks autonomously, using data and algorithms to  │
│  make decisions and interact with their environment. These agents can range from simple automated tools to      │
│  complex systems capable of learning and adapting over time. At their core, AI agents are characterized by      │
│  their ability to perceive their environment, process information, and act accordingly.                         │
│                                                                                                                 │
│  The functionality of AI agents is rooted in their capacity to execute tasks without direct human               │
│  intervention. This autonomy is enabled by sophisticated algorithms that allow agents to analyze data,          │
│  recognize patterns, and make informed decisions. For instance, a simple AI agent might be programmed to sort   │
│  emails based on predefined criteria, while a more complex agent could be designed to manage a smart home       │
│  system, adjusting temperature, lighting, and security settings based on the occupants' preferences and         │
│  habits.                                                                                                        │
│                                                                                                                 │
│  ## Examples of AI Agents Across Industries                                                                     │
│  AI agents are being increasingly adopted across various industries, transforming the way businesses operate    │
│  and deliver services. In customer service, AI-powered chatbots are used to provide immediate support,          │
│  handling queries, and resolving issues. In healthcare, AI agents are employed to monitor patient conditions,   │
│  predict potential health risks, and personalize treatment plans. The finance sector utilizes AI agents for     │
│  fraud detection, risk assessment, and portfolio management, enhancing security and efficiency.                 │
│                                                                                                                 │
│  In the automotive industry, AI agents are crucial for the development of autonomous vehicles, enabling cars    │
│  to navigate, make decisions, and avoid obstacles. Similarly, in manufacturing, AI agents are used to optimize  │
│  production processes, predict maintenance needs, and i

...

/home/crs/10Academy/llm-projects/multiAgent/.venv/lib/python3.10/site-packages/httpx/_models.py:408: 
DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(

2025-07-28 20:44:51,902 - 131823051266944 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-28 20:44:55,564 - 131823051266944 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
2025-07-28 20:44:58,506 - 131823051266944 - rag_storage.py-rag_storage:103 - ERROR: Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bc9f8706-e0e7-4ced-87d9-62fa8f10937f                                                                     │
│  Agent: AI agents Senior Researcher                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c1e2ac8a-3131-4788-8bd6-39f6cbbb5a99                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Thought                                                                                       │
│  To address the task, I will first define and explain AI agents in a formal and easy-to-understand manner.      │
│  Then, I will identify the next big trend in AI agent technologies, discussing their pros and cons. I will      │
│  provide a detailed report in markdown format, including examples of AI agents across various industries,       │
│  technical details on Multi-Agent Systems, and potential future applications beyond current trends.             │
│                                                                                                                 │
│  ## Final Answer                                                                                                │
│  # AI Agents: Definition and Explanation                                                                        │
│  AI agents are software programs designed to perform specific tasks autonomously, using data and algorithms to  │
│  make decisions and interact with their environment. These agents can range from simple automated tools to      │
│  complex systems capable of learning and adapting over time. At their core, AI agents are characterized by      │
│  their ability to perceive their environment, process information, and act accordingly.                         │
│                                                                                                                 │
│  The functionality of AI agents is rooted in their capacity to execute tasks without direct human               │
│  intervention. This autonomy is enabled by sophisticated algorithms that allow agents to analyze data,          │
│  recognize patterns, and make informed decisions. For instance, a simple AI agent might be programmed to sort   │
│  emails based on predefined criteria, while a more complex agent could be designed to manage a smart home       │
│  system, adjusting temperature, lighting, and security settings based on the occupants' preferences and         │
│  habits.                                                                                                        │
│                                                                                                                 │
│  ## Examples of AI Agents Across Industries                                                                     │
│  AI agents are being increasingly adopted across various industries, transforming the way businesses operate    │
│  and deliver services. In customer service, AI-powered chatbots are used to provide immediate support,          │
│  handling queries, and resolving issues. In healthcare, AI agents are employed to monitor patient conditions,   │
│  predict potential health risks, and personalize treatment plans. The finance sector utilizes AI agents for     │
│  fraud detection, risk assessment, and portfolio management, enhancing security and efficiency.                 │
│                                                                                                                 │
│  In the automotive industry, AI agents are crucial for the development of autonomous vehicles, enabling cars    │
│  to navigate, make decisions, and avoid obstacles. Sim

## Thought
To address the task, I will first define and explain AI agents in a formal and easy-to-understand manner. Then, I will identify the next big trend in AI agent technologies, discussing their pros and cons. I will provide a detailed report in markdown format, including examples of AI agents across various industries, technical details on Multi-Agent Systems, and potential future applications beyond current trends.

## Final Answer
# AI Agents: Definition and Explanation
AI agents are software programs designed to perform specific tasks autonomously, using data and algorithms to make decisions and interact with their environment. These agents can range from simple automated tools to complex systems capable of learning and adapting over time. At their core, AI agents are characterized by their ability to perceive their environment, process information, and act accordingly.

The functionality of AI agents is rooted in their capacity to execute tasks without direct human intervention. This autonomy is enabled by sophisticated algorithms that allow agents to analyze data, recognize patterns, and make informed decisions. For instance, a simple AI agent might be programmed to sort emails based on predefined criteria, while a more complex agent could be designed to manage a smart home system, adjusting temperature, lighting, and security settings based on the occupants' preferences and habits.

## Examples of AI Agents Across Industries
AI agents are being increasingly adopted across various industries, transforming the way businesses operate and deliver services. In customer service, AI-powered chatbots are used to provide immediate support, handling queries, and resolving issues. In healthcare, AI agents are employed to monitor patient conditions, predict potential health risks, and personalize treatment plans. The finance sector utilizes AI agents for fraud detection, risk assessment, and portfolio management, enhancing security and efficiency.

In the automotive industry, AI agents are crucial for the development of autonomous vehicles, enabling cars to navigate, make decisions, and avoid obstacles. Similarly, in manufacturing, AI agents are used to optimize production processes, predict maintenance needs, and improve product quality. These examples illustrate the versatility and potential of AI agents to revolutionize different sectors.

## Multi-Agent Systems: Technical Details
Multi-Agent Systems (MAS) represent a significant advancement in AI agent technology, involving multiple agents that interact with each other to achieve complex goals. These systems are designed to tackle problems that are too intricate for a single agent to solve. MAS are characterized by their ability to facilitate cooperation, coordination, and negotiation among agents, enabling them to work together seamlessly.

The implementation of MAS involves several technical challenges, including designing effective communication protocols, managing agent interactions, and ensuring system scalability. Agents in MAS must be able to understand and respond to the actions of other agents, requiring sophisticated algorithms and coordination mechanisms. For instance, in a smart grid application, multiple AI agents might be used to manage different aspects of energy distribution, such as supply, demand, and storage, working together to optimize energy efficiency.

## Next Big Trend in AI Agents: Autonomous Collaborative Systems
The next big trend in AI agent technologies is the development of Autonomous Collaborative Systems, where AI agents are designed to work together in complex ecosystems, enhancing their capabilities and achieving outcomes that are beyond the reach of individual agents. This trend is driven by advancements in areas like edge computing, 5G networks, and the Internet of Things (IoT), enabling more sophisticated and interconnected AI agent systems.

These Autonomous Collaborative Systems have the potential to transform various domains, from smart cities to industrial automation. For example, in a smart city context, AI agents could collaborate to manage traffic flow, public safety, and energy usage, creating a more efficient and sustainable urban environment. However, this trend also comes with challenges, such as ensuring the security and integrity of the collaborative systems, managing the complexity of agent interactions, and addressing ethical considerations related to autonomy and decision-making.

## Pros and Cons of Autonomous Collaborative Systems
The pros of Autonomous Collaborative Systems include enhanced efficiency, improved decision-making, and the ability to tackle complex problems that are currently unsolvable. These systems can also lead to significant advancements in areas like healthcare, transportation, and energy management. However, there are also cons to consider, such as the potential for increased dependency on technology, challenges in ensuring system reliability and security, and the need for new regulatory frameworks to govern the use of autonomous AI agents.

In conclusion, AI agents are poised to play a crucial role in shaping the future of various industries and domains. The emergence of Autonomous Collaborative Systems represents a significant trend in AI agent technologies, offering both opportunities and challenges. As these systems continue to evolve, it is essential to address the associated technical, ethical, and regulatory issues to fully harness their potential.